In [2]:
import pandas as pd
import numpy as np
import scipy.stats as stats

## Loading Data

In [19]:
#import all csvs I'll use from the data folder into dataframes.
#each dataframe is accessed via datasets[<name>], with names 
#being the same as those in the list documents
path = './data/'
documents = ['diversity_school', 'salary_potential', 'tuition_income',
'historical_tuition', 'tuition_cost', 'table', 'cpi']
datasets = {}

for doc in documents:
    filename = path + doc + '.csv'
    df = pd.read_csv(filename, sep=',', header=0)
    datasets[doc] = df

In [20]:
#example of how to access a dataframe
datasets['cpi'].head(3)

,Year,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,Annual,HALF1,HALF2
0,1985,105.5,106.0,106.4,106.9,107.3,107.6,107.8,108.0,108.3,108.7,109.0,109.3,107.6,106.6,108.5
1,1986,109.6,109.3,108.8,108.6,108.9,109.5,109.5,109.7,110.2,110.3,110.4,110.5,109.6,109.1,110.1
2,1987,111.2,111.6,112.1,112.7,113.1,113.5,113.8,114.4,115.0,115.3,115.4,115.4,113.6,112.4,114.9


## Data Cleaning

In [4]:
#the first year '1985-19862 is odd due to a footnote character from the webpage
datasets['table']['year'].unique()

array(['1985–862', '1995–96', '2000–01', '2001–02', '2002–03', '2003–04',
       '2004–05', '2005–06', '2006–07', '2007–08', '2008–09', '2009–10',
       '2010–11', '2011–12', '2012–13', '2013–14', '2014–15', '2015–16',
       '2016–17', '2017–18', '2018–19'], dtype=object)

In [5]:
#the odd value is cleaned, and the character '–' is replaced with '-' so that
#the year columns between historical_tuition and table match
datasets['table'].loc[datasets['table']['year'] == '1985–862', 'year'] = '1985-86'
datasets['table']['year'] = datasets['table']['year'].str.replace('–','-')
datasets['table'].head(1)

,type,year,Constant_All,Constant_4-Year,Constant_2-Year,Contemporary_All,Contemporary_4-Year,Contemporary_2-Year
0,All Institutions,1985-86,11369,12811,7836,4885,5504,3367


In [7]:
#table is in a format where I'm having trouble directly grabbing the tuition values
#I want, so instead I extract columns into seperate dataframes, and recombine them
#so that the format better matches historical_tuition.

const_all = datasets['table'][['type', 'year', 'Constant_All']]
const_all = const_all.rename(columns = {'Constant_All': 'tuition_2'})
const_all['tuition_type'] = 'All Constant'

const_4 = datasets['table'][['type', 'year', 'Constant_4-Year']]
const_4 = const_4.rename(columns = {'Constant_4-Year': 'tuition_2'})
const_4['tuition_type'] = '4 Year Constant'

const_2 = datasets['table'][['type', 'year', 'Constant_2-Year']]
const_2 = const_2.rename(columns = {'Constant_2-Year': 'tuition_2'})
const_2['tuition_type'] = '2 Year Constant'

curr_all = datasets['table'][['type', 'year', 'Contemporary_All']]
curr_all = curr_all.rename(columns = {'Contemporary_All': 'tuition_2'})
curr_all['tuition_type'] = 'All Current'

curr_4 = datasets['table'][['type', 'year', 'Contemporary_4-Year']]
curr_4 = curr_4.rename(columns = {'Contemporary_4-Year': 'tuition_2'})
curr_4['tuition_type'] = '4 Year Current'

curr_2 = datasets['table'][['type', 'year', 'Contemporary_2-Year']]
curr_2 = curr_2.rename(columns = {'Contemporary_2-Year': 'tuition_2'})
curr_2['tuition_type'] = '2 Year Current'

#The concats could be done in one step, but I want to check the shape
#to make sure nothing's dissappearing
const = pd.concat([const_all, const_2,const_4], ignore_index=True)
curr = pd.concat([curr_all, curr_2, curr_4], ignore_index=True)
print(const.shape)
print(curr.shape)

all_table_rows = pd.concat([const, curr], ignore_index=True)
print(all_table_rows.shape)

(186, 4)
(186, 4)
(372, 4)


In [8]:
#merge dataframes and look for NAs
hist_tuition = pd.merge(datasets['historical_tuition'], all_table_rows, how='left', on=['type','year','tuition_type'])
hist_tuition.isnull().sum().sum()

0

In [10]:
hist_tuition.head()

,type,year,tuition_type,tuition_cost,tuition_2
0,All Institutions,1985-86,All Constant,10893,11369
1,All Institutions,1985-86,4 Year Constant,12274,12811
2,All Institutions,1985-86,2 Year Constant,7508,7836
3,All Institutions,1985-86,All Current,4885,4885
4,All Institutions,1985-86,4 Year Current,5504,5504


In [18]:
#I make a third column with the average of the two tuition values
hist_tuition['avg_tuition'] = ((hist_tuition['tuition_cost'] + hist_tuition['tuition_2'])/2).astype(int)
hist_tuition.head(3)

,type,year,tuition_type,tuition_cost,tuition_2,avg_tuition
0,All Institutions,1985-86,All Constant,10893,11369,11131
1,All Institutions,1985-86,4 Year Constant,12274,12811,12542
2,All Institutions,1985-86,2 Year Constant,7508,7836,7672


## Question 1 - How has Diversity Changed over Time

## Question 2 - How has Rate of Tuition Changed with Rate of Inflation?

In [ ]:
#